In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import sparse

# Constructing hypergraphs and corresponding comparison graphs

In [2]:
# We want the nonzero rows of W and R to sum to 1 
def row_normalize(X):
    Y = np.matrix.copy(X)
    for i in range(len(Y)):
        row = Y[i]
        row_sum = np.sum(row)
        if row_sum != 0:
            Y[i] = Y[i]/row_sum   
    return Y

In [3]:
# E: number of users
# V: number of items
# p: chance certain user watches certain movie
# e1: chance edge weight is flipped
# e2: chance user rating is flipped
# e3: chance user rating is erased
def make_hypergraphs(E, V, p, e1, e2, e3, random_seed):
    np.random.seed(random_seed)
    
    n = int(E/3)
    m = int(V/3)
    
    W = np.random.rand(V, E) # hyperedge-weight matrix, |V| x |E|, each row corresponds to a movie. 
    num_pairs = 0

    for i in range(V):
        for j in range(E):
            if W[i][j] < p:
                W[i][j] = 1

                num_pairs += 1
            else:
                W[i][j] = 0
                
    W1 = np.copy(W)
    W2 = np.copy(W)
    W3 = np.copy(W)
                
    for j in range(E):
        W1_multiplier = 100 if j < n else 1
        W2_multiplier = 100 if j < 2 * n and j >= n else 1
        W3_multiplier = 100 if j >= 2 * n else 1
       
        err = np.random.rand(1)
        if err < e1:
            W1_multiplier = 101 - W1_multiplier 
            W2_multiplier = 101 - W2_multiplier
            W3_multiplier = 101 - W3_multiplier
            
        W1[:,j] *= W1_multiplier
        W2[:,j] *= W2_multiplier
        W3[:,j] *= W3_multiplier

                
    R = np.zeros((E, V)) # edge-dependent vertex-weight matrix, |E| x |V|, each row corresponds to a user.
    true_R = np.zeros((E, V)) # true ratings

    for i in range(V):
        for j in range(E):
            if W[i][j] == 1:
                if i // n == j // m:
                    true_R[j][i] = 5
                else:
                    true_R[j][i] = 1

                err = np.random.rand(1)
                if err < e2:
                    R[j][i] = 6 - true_R[j][i]
                elif err >= e2 and err < e2 + e3:
                    W[i][j] == 0
                    # do nothing as R[j][i] is already 0
                else:
                    R[j][i] = true_R[j][i]
                    
    WG1 = np.zeros((V+E, num_pairs)) # hyperedge weight matrix, weights are ratings x 10 IF user likes category
    WG2 = np.zeros((V+E, num_pairs)) # otherwise 1
    WG3 = np.zeros((V+E, num_pairs))

    RG = np.zeros((num_pairs, V+E)) # edge-dependent vertex weight matrix, weights are 1

    curr_edge_index = 0 

    for i in range(V):
        for j in range(E):
            if R[j][i] != 0:
                # movie index = i
                # user index = V+j

                RG[curr_edge_index][V+j] = 1
                RG[curr_edge_index][i] = 1

                WG1[V+j][curr_edge_index] = W1[i][j] * R[j][i]
                WG1[i][curr_edge_index] = WG1[V+j][curr_edge_index]

                WG2[V+j][curr_edge_index] = W2[i][j] * R[j][i]
                WG2[i][curr_edge_index] = WG2[V+j][curr_edge_index]

                WG3[V+j][curr_edge_index] = W3[i][j] * R[j][i] 
                WG3[i][curr_edge_index] = WG3[V+j][curr_edge_index]

                curr_edge_index += 1
                
                
    W1s = sparse.csr_matrix(row_normalize(W1))
    W2s = sparse.csr_matrix(row_normalize(W2))
    W3s = sparse.csr_matrix(row_normalize(W3))
    Rs = sparse.csr_matrix(row_normalize(R))

    WG1s = sparse.csr_matrix(row_normalize(WG1))
    WG2s = sparse.csr_matrix(row_normalize(WG2))
    WG3s = sparse.csr_matrix(row_normalize(WG3))
    RGs = sparse.csr_matrix(row_normalize(RG))

    # create prob trans matrices
    P1 = np.transpose(W1s.dot(Rs))
    P2 = np.transpose(W2s.dot(Rs))
    P3 = np.transpose(W3s.dot(Rs))

    PG1 = np.transpose(WG1s.dot(RGs))
    PG2 = np.transpose(WG2s.dot(RGs))
    PG3 = np.transpose(WG3s.dot(RGs))

    return P1, P2, P3, PG1, PG2, PG3, R, true_R

# Computing personalized PageRank rankings

In [4]:
# given probability transition matrix P
# where P_{v,w} = Prob(w -> v)
# find pagerank scores with restart probability r
def compute_pr(P, r, n, home, eps=1e-8):
    
    x = np.ones(n) / n*1.0

    flag = True
    t=0
        
    while flag:
        x_new = (1-r)*P*x

        x_new = x_new + home * r 
        
        if np.linalg.norm(x_new - x,ord=1) < eps and t > 100:
            flag = False
        t=t+1
        x = x_new
    
    return x

In [5]:
def get_rankings(E, V, R, P1, P2, P3, PG1, PG2, PG3, r):
    
    rankings_hg = np.zeros((E, V)) # each row corresponds to a user. 
    rankings_g = np.zeros((E, V)) # each row corresponds to a user. 
    n = int(E/3)
    
    for i in range(E):

        if i < n:
            P = P1
            PG = PG1
        elif i < 2 * n:
            P = P2
            PG = PG2
        else: 
            P = P3
            PG = PG3

        # personalize the algorithm by restarting at any of the movies a certain user originally watched
        home_hg = np.zeros(V)

        for j in range(V):
            if R[i][j] != 0:
                home_hg[j] = 1

        if np.sum(home_hg) > 0:
            home_hg = home_hg / np.sum(home_hg)

        rankings_hg[i,:] = compute_pr(P, r, V, home_hg).flatten()

        # same process for the graph
        home_g = np.zeros(V+E)
        home_g[V+i] = 1

        curr_rankings_g = compute_pr(PG, r, V+E, home_g).flatten()
        rankings_g[i,:] = curr_rankings_g[:V]
        
    return rankings_hg, rankings_g

# Evaluating rankings

In [6]:
# Source: https://www.aaai.org/Papers/IJCAI/2007/IJCAI07-444.pdf
def calc_avg_doa(num_users, num_movies, ratings, rankings):
    
    n = num_users/3
    m = num_movies/3
    
    total_pairs = 0
    correct_pairs = 0
    
    # All pairs of movies. 
    for i in range(num_movies):
        for j in range(i+1, num_movies):
            for user in range(num_users):

                if i // m != j // m:
                    if user // n == i // m:
                        total_pairs += 1
                        if rankings[user][i] > rankings[user][j]:
                            correct_pairs += 1
                    elif user // n == j // m:
                        total_pairs += 1
                        if rankings[user][i] < rankings[user][j]:
                            correct_pairs += 1
       
    if total_pairs == 0:
        return -1
    return correct_pairs/total_pairs

def calc_avg_udoa(num_users, num_movies, ratings, rankings):
    
    n = num_users/3
    m = num_movies/3
    
    total_pairs = 0
    correct_pairs = 0
    
    # All pairs of movies. 
    for i in range(num_movies):
        for j in range(i+1, num_movies):
            for user in range(num_users):
                
                if ratings[user][i] == 0 and ratings[user][j] == 0:
                    if i // m != j // m:
                        if user // n == i // m:
                            total_pairs += 1
                            if rankings[user][i] > rankings[user][j]:
                                correct_pairs += 1
                        elif user // n == j // m:
                            total_pairs += 1
                            if rankings[user][i] < rankings[user][j]:
                                correct_pairs += 1
       
    if total_pairs == 0:
        return -1
    return correct_pairs/total_pairs

In [7]:
def do_everything(E, V, p, e1, e2, e3, random_seed):
    
    n = int(E/3)
    m = int(V/3)
    
    P1, P2, P3, PG1, PG2, PG3, R, true_R = make_hypergraphs(E, V, p, e1, e2, e3, random_seed)


    rankings_hg, rankings_g = get_rankings(E, V, R, P1, P2, P3, PG1, PG2, PG3, 0.15)
    
    avgdoa_hg = calc_avg_doa(E, V, true_R, rankings_hg)
    avgdoa_g = calc_avg_doa(E, V, true_R, rankings_g)
    avgudoa_hg = calc_avg_udoa(E, V, true_R, rankings_hg)
    avgudoa_g = calc_avg_udoa(E, V, true_R, rankings_g)

    return avgdoa_hg, avgdoa_g, avgudoa_hg, avgudoa_g

In [8]:
# To remove the effects of randomness, average results generated by many random seeds. 
def do_everything_n_times(E, V, p, e1, e2, e3, n):
    print("n=%d, p=%.3f, e1=%.3f, e2=%.3f, e3=%.3f" % (n, p, e1, e2, e3))
    
    if n == 0:
        return
    
    d_hg = 0
    d_g = 0
    ud_hg = 0
    ud_g = 0
    
    d_hgs = np.zeros(n)
    d_gs = np.zeros(n)
    ud_hgs = np.zeros(n)
    ud_gs = np.zeros(n)
    
    for i in range(n):
        d1, d2, ud1, ud2 = do_everything(E, V, p, e1, e2, e3, i)
        d_hgs[i] = d1
        d_gs[i] = d2
        ud_hgs[i] = ud1
        ud_gs[i] = ud2
    
    return d_hgs, d_gs, ud_hgs, ud_gs

# Running

In [9]:
n = 50
m = 50
    
E = 3 * n # number of "users"
V = 3 * m # number of "movies

In [10]:
probs = np.linspace(0, 1, num=11)
num_probs = len(probs)

e1_hg = np.zeros(num_probs)
e1_g = np.zeros(num_probs)
e2_hg = np.zeros(num_probs)
e2_g = np.zeros(num_probs)
e3_hg = np.zeros(num_probs)
e3_g = np.zeros(num_probs)

e1_hg_u = np.zeros(num_probs)
e1_g_u = np.zeros(num_probs)
e2_hg_u = np.zeros(num_probs)
e2_g_u = np.zeros(num_probs)
e3_hg_u = np.zeros(num_probs)
e3_g_u = np.zeros(num_probs)

In [11]:
# All this is just to draw error bars on the plot. 
# Was it efficient? I don't know. I shoved it into the appendix. 
e1_hg_errs = np.zeros((2, num_probs))
e1_g_errs = np.zeros((2, num_probs))
e2_hg_errs = np.zeros((2, num_probs))
e2_g_errs = np.zeros((2, num_probs))
e3_hg_errs = np.zeros((2, num_probs))
e3_g_errs = np.zeros((2, num_probs))

e1_hg_u_errs = np.zeros((2, num_probs))
e1_g_u_errs = np.zeros((2, num_probs))
e2_hg_u_errs = np.zeros((2, num_probs))
e2_g_u_errs = np.zeros((2, num_probs))
e3_hg_u_errs = np.zeros((2, num_probs))
e3_g_u_errs = np.zeros((2, num_probs))

In [12]:
# apologies for the code you are about to see
d_hgs, d_gs, ud_hgs, ud_gs = do_everything_n_times(E, V, 0.15, 0, 0, 0, 10)

##

e1_hg[0] = np.average(d_hgs)
e1_hg_errs[0,0] = np.average(d_hgs) - np.amin(d_hgs)
e1_hg_errs[1,0] = np.amax(d_hgs) - np.average(d_hgs)

e1_g[0] = np.average(d_gs)
e1_g_errs[0,0] = np.average(d_gs) - np.amin(d_gs)
e1_g_errs[1,0] = np.amax(d_gs) - np.average(d_gs)

e1_hg_u[0] = np.average(ud_hgs)
e1_hg_u_errs[0,0] = np.average(ud_hgs) - np.amin(ud_hgs)
e1_hg_u_errs[1,0] = np.amax(ud_hgs) - np.average(ud_hgs)

e1_g_u[0] = np.average(ud_gs)
e1_g_u_errs[0,0] = np.average(ud_gs) - np.amin(ud_gs)
e1_g_u_errs[1,0] = np.amax(ud_gs) - np.average(ud_gs)

##

e2_hg[0] = np.average(d_hgs)
e2_hg_errs[0,0] = np.average(d_hgs) - np.amin(d_hgs)
e2_hg_errs[1,0] = np.amax(d_hgs) - np.average(d_hgs)

e2_g[0] = np.average(d_gs)
e2_g_errs[0,0] = np.average(d_gs) - np.amin(d_gs)
e2_g_errs[1,0] = np.amax(d_gs) - np.average(d_gs)

e2_hg_u[0] = np.average(ud_hgs)
e2_hg_u_errs[0,0] = np.average(ud_hgs) - np.amin(ud_hgs)
e2_hg_u_errs[1,0] = np.amax(ud_hgs) - np.average(ud_hgs)

e2_g_u[0] = np.average(ud_gs)
e2_g_u_errs[0,0] = np.average(ud_gs) - np.amin(ud_gs)
e2_g_u_errs[1,0] = np.amax(ud_gs) - np.average(ud_gs)

##

e3_hg[0] = np.average(d_hgs)
e3_hg_errs[0,0] = np.average(d_hgs) - np.amin(d_hgs)
e3_hg_errs[1,0] = np.amax(d_hgs) - np.average(d_hgs)

e3_g[0] = np.average(d_gs)
e3_g_errs[0,0] = np.average(d_gs) - np.amin(d_gs)
e3_g_errs[1,0] = np.amax(d_gs) - np.average(d_gs)

e3_hg_u[0] = np.average(ud_hgs)
e3_hg_u_errs[0,0] = np.average(ud_hgs) - np.amin(ud_hgs)
e3_hg_u_errs[1,0] = np.amax(ud_hgs) - np.average(ud_hgs)

e3_g_u[0] = np.average(ud_gs)
e3_g_u_errs[0,0] = np.average(ud_gs) - np.amin(ud_gs)
e3_g_u_errs[1,0] = np.amax(ud_gs) - np.average(ud_gs)

n=10, p=0.150, e1=0.000, e2=0.000, e3=0.000


In [13]:
for i in range(1, num_probs):
    prob = probs[i]
    
    ##
    
    d_hgs, d_gs, ud_hgs, ud_gs = do_everything_n_times(E, V, 0.15, prob, 0, 0, 10)
    
    e1_hg[i] = np.average(d_hgs)
    e1_hg_errs[0,i] = np.average(d_hgs) - np.amin(d_hgs)
    e1_hg_errs[1,i] = np.amax(d_hgs) - np.average(d_hgs)

    e1_g[i] = np.average(d_gs)
    e1_g_errs[0,i] = np.average(d_gs) - np.amin(d_gs)
    e1_g_errs[1,i] = np.amax(d_gs) - np.average(d_gs)

    e1_hg_u[i] = np.average(ud_hgs)
    e1_hg_u_errs[0,i] = np.average(ud_hgs) - np.amin(ud_hgs)
    e1_hg_u_errs[1,i] = np.amax(ud_hgs) - np.average(ud_hgs)

    e1_g_u[i] = np.average(ud_gs)
    e1_g_u_errs[0,i] = np.average(ud_gs) - np.amin(ud_gs)
    e1_g_u_errs[1,i] = np.amax(ud_gs) - np.average(ud_gs)
    
    ##
    
    d_hgs, d_gs, ud_hgs, ud_gs = do_everything_n_times(E, V, 0.15, 0, prob, 0, 10)
    
    e2_hg[i] = np.average(d_hgs)
    e2_hg_errs[0,i] = np.average(d_hgs) - np.amin(d_hgs)
    e2_hg_errs[1,i] = np.amax(d_hgs) - np.average(d_hgs)

    e2_g[i] = np.average(d_gs)
    e2_g_errs[0,i] = np.average(d_gs) - np.amin(d_gs)
    e2_g_errs[1,i] = np.amax(d_gs) - np.average(d_gs)

    e2_hg_u[i] = np.average(ud_hgs)
    e2_hg_u_errs[0,i] = np.average(ud_hgs) - np.amin(ud_hgs)
    e2_hg_u_errs[1,i] = np.amax(ud_hgs) - np.average(ud_hgs)

    e2_g_u[i] = np.average(ud_gs)
    e2_g_u_errs[0,i] = np.average(ud_gs) - np.amin(ud_gs)
    e2_g_u_errs[1,i] = np.amax(ud_gs) - np.average(ud_gs)
    
    ##
    
    d_hgs, d_gs, ud_hgs, ud_gs = do_everything_n_times(E, V, 0.15, 0, 0, prob, 10)
    
    e3_hg[i] = np.average(d_hgs)
    e3_hg_errs[0,i] = np.average(d_hgs) - np.amin(d_hgs)
    e3_hg_errs[1,i] = np.amax(d_hgs) - np.average(d_hgs)

    e3_g[i] = np.average(d_gs)
    e3_g_errs[0,i] = np.average(d_gs) - np.amin(d_gs)
    e3_g_errs[1,i] = np.amax(d_gs) - np.average(d_gs)

    e3_hg_u[i] = np.average(ud_hgs)
    e3_hg_u_errs[0,i] = np.average(ud_hgs) - np.amin(ud_hgs)
    e3_hg_u_errs[1,i] = np.amax(ud_hgs) - np.average(d_gs)

    e3_g_u[i] = np.average(ud_gs)
    e3_g_u_errs[0,i] = np.average(ud_gs) - np.amin(ud_gs)
    e3_g_u_errs[1,i] = np.amax(ud_gs) - np.average(ud_gs)

    print()

n=10, p=0.150, e1=0.100, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.100, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.100

n=10, p=0.150, e1=0.200, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.200, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.200

n=10, p=0.150, e1=0.300, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.300, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.300

n=10, p=0.150, e1=0.400, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.400, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.400

n=10, p=0.150, e1=0.500, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.500, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.500

n=10, p=0.150, e1=0.600, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.600, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.600

n=10, p=0.150, e1=0.700, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.700, e3=0.000
n=10, p=0.150, e1=0.000, e2=0.000, e3=0.700

n=10, p=0.150, e1=0.800, e2=0.000, e3=0.000
n=10, p=0.150, e1=0.000, 

In [14]:
print("%s = %s" % ("e1_hg",  np.array2string(e1_hg, separator=', ')))
print("%s = %s" % ("e1_g",  np.array2string(e1_g, separator=', ')))
print("%s = %s" % ("e1_hg_u",  np.array2string(e1_hg_u, separator=', ')))
print("%s = %s" % ("e1_g_u",  np.array2string(e1_g_u, separator=', ')))

print()

print("%s = %s" % ("e1_hg_errs",  np.array2string(e1_hg_errs, separator=', ')))
print("%s = %s" % ("e1_g_errs",  np.array2string(e1_g_errs, separator=', ')))
print("%s = %s" % ("e1_hg_u_errs",  np.array2string(e1_hg_u_errs, separator=', ')))
print("%s = %s" % ("e1_g_u_errs",  np.array2string(e1_g_u_errs, separator=', ')))

e1_hg = [0.94940453, 0.89848693, 0.82704067, 0.74324467, 0.62540733, 0.49242613,
 0.36750067, 0.26243267, 0.18223187, 0.13549627, 0.10881147]
e1_g = [0.877446  , 0.85307467, 0.8157572 , 0.76803493, 0.69882947, 0.61151493,
 0.5110576 , 0.40563053, 0.2995676 , 0.2154764 , 0.15916573]
e1_hg_u = [0.9945177 , 0.9662201 , 0.90067333, 0.80509172, 0.65915388, 0.48615968,
 0.32233435, 0.1860325 , 0.08533305, 0.03223491, 0.00824078]
e1_g_u = [0.99270582, 0.96751921, 0.91912597, 0.85440711, 0.75933224, 0.63848409,
 0.49910307, 0.35271921, 0.20560955, 0.08893313, 0.01083144]

e1_hg_errs = [[0.01824987, 0.0245496 , 0.04328867, 0.08761533, 0.073026  , 0.06116747,
  0.08858067, 0.07097267, 0.0241692 , 0.01025493, 0.00530747],
 [0.01195547, 0.02257173, 0.043278  , 0.085382  , 0.06398867, 0.0497712 ,
  0.04296067, 0.07141533, 0.02866547, 0.01475173, 0.00981253]]
e1_g_errs = [[0.01823533, 0.02480933, 0.01911987, 0.0509216 , 0.0497268 , 0.03041493,
  0.0710696 , 0.0858292 , 0.04242093, 0.02504307, 0.0066

In [15]:
print("%s = %s" % ("e2_hg",  np.array2string(e2_hg, separator=', ')))
print("%s = %s" % ("e2_g",  np.array2string(e2_g, separator=', ')))
print("%s = %s" % ("e2_hg_u",  np.array2string(e2_hg_u, separator=', ')))
print("%s = %s" % ("e2_g_u",  np.array2string(e2_g_u, separator=', ')))

print()

print("%s = %s" % ("e2_hg_errs",  np.array2string(e2_hg_errs, separator=', ')))
print("%s = %s" % ("e2_g_errs",  np.array2string(e2_g_errs, separator=', ')))
print("%s = %s" % ("e2_hg_u_errs",  np.array2string(e2_hg_u_errs, separator=', ')))
print("%s = %s" % ("e2_g_u_errs",  np.array2string(e2_g_u_errs, separator=', ')))

e2_hg = [0.94940453, 0.90492933, 0.84245187, 0.75246173, 0.6328692 , 0.50107053,
 0.3698528 , 0.2556104 , 0.1741616 , 0.12265107, 0.0921452 ]
e2_g = [0.877446  , 0.8507576 , 0.8032096 , 0.72592107, 0.6190004 , 0.5016388 ,
 0.38112947, 0.2744768 , 0.19630067, 0.14760333, 0.12108453]
e2_hg_u = [0.9945177 , 0.97265209, 0.92024541, 0.82033433, 0.67180449, 0.50135154,
 0.33004967, 0.18215345, 0.08144394, 0.02601919, 0.00355345]
e2_g_u = [0.99270582, 0.965318  , 0.90493743, 0.80225218, 0.65915387, 0.50204616,
 0.34071286, 0.19799605, 0.09427907, 0.03231989, 0.0049148 ]

e2_hg_errs = [[0.01824987, 0.013048  , 0.01631453, 0.02362707, 0.0275172 , 0.0280532 ,
  0.02542747, 0.0224224 , 0.01360293, 0.00773773, 0.0042532 ],
 [0.01195547, 0.00808667, 0.0146908 , 0.0244596 , 0.03230813, 0.03118413,
  0.02355253, 0.01944693, 0.0106584 , 0.00906093, 0.00494813]]
e2_g_errs = [[0.01823533, 0.01587227, 0.0183976 , 0.02362373, 0.02920707, 0.0294388 ,
  0.02494413, 0.02507147, 0.01520867, 0.00879667, 0.0065

In [16]:
print("%s = %s" % ("e3_hg",  np.array2string(e3_hg, separator=', ')))
print("%s = %s" % ("e3_g",  np.array2string(e3_g, separator=', ')))
print("%s = %s" % ("e3_hg_u",  np.array2string(e3_hg_u, separator=', ')))
print("%s = %s" % ("e3_g_u",  np.array2string(e3_g_u, separator=', ')))

print()

print("%s = %s" % ("e3_hg_errs",  np.array2string(e3_hg_errs, separator=', ')))
print("%s = %s" % ("e3_g_errs",  np.array2string(e3_g_errs, separator=', ')))
print("%s = %s" % ("e3_hg_u_errs",  np.array2string(e3_hg_u_errs, separator=', ')))
print("%s = %s" % ("e3_g_u_errs",  np.array2string(e3_g_u_errs, separator=', ')))

e3_hg = [0.94940453, 0.9404888 , 0.93150493, 0.922684  , 0.9099368 , 0.89168893,
 0.8604868 , 0.80022707, 0.69785333, 0.53871467, 0.        ]
e3_g = [0.877446  , 0.88087853, 0.8834332 , 0.88397053, 0.87412773, 0.84958547,
 0.80424573, 0.72571493, 0.61913453, 0.51426307, 0.        ]
e3_hg_u = [0.9945177 , 0.99160386, 0.98671406, 0.97918676, 0.96451392, 0.93961596,
 0.89910315, 0.82580012, 0.70959501, 0.53975818, 0.        ]
e3_g_u = [0.99270582, 0.98899914, 0.98204131, 0.97087387, 0.94664799, 0.90568451,
 0.84300435, 0.74679472, 0.6263997 , 0.51489551, 0.        ]

e3_hg_errs = [[0.01824987, 0.0129768 , 0.01221693, 0.01318133, 0.02373147, 0.0278636 ,
  0.0342548 , 0.04230707, 0.046896  , 0.03310667, 0.        ],
 [0.01195547, 0.0086432 , 0.0119204 , 0.015464  , 0.01747653, 0.01972307,
  0.02135853, 0.0414236 , 0.05126267, 0.03610667, 0.        ]]
e3_g_errs = [[0.01823533, 0.01662787, 0.01486787, 0.0133212 , 0.02110773, 0.0249988 ,
  0.02719907, 0.02774027, 0.0316332 , 0.0160604 , 0.    